In [2]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

# Now access API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API key not found. check .env file")


print("✓ Libraries imported successfully")
print(f"✓ OpenAI library version: {openai.__version__}")

✓ Libraries imported successfully
✓ OpenAI library version: 1.109.1


In [3]:
# Cell 3: Test API connection
# This simple test verifies your API key works

from openai import OpenAI

client = OpenAI(api_key=api_key)

try:
    model_to_use = "gpt-4"
    response = client.chat.completions.create(
        model=model_to_use,
        messages=[
            {"role":"user", "content": "Say 'API connection Successful' If you can read this."}
        ],
        max_tokens=50

    )

    print(f"\n✓ API Connection Test (model={model_to_use}):")
    print(response.choices[0].message.content)
    print("\n✓ Setup complete! Ready to build the agent.")
    
except Exception as e:
    print(f"⚠ GPT-4 not accessible: {e}")
    print("Trying GPT-3.5-turbo instead...")
    print(f"✗ Error: {e}")
    print("Please check your API key and try again.")


    try:
        model_to_use = "gpt-3.5-turbo"
        response = client.chat.completions.create(
            model=model_to_use,
            messages=[{"role": "user", "content": "Say 'API connection Successful' if you can read this."}],
            max_tokens=50
        )
        print(f"\n✓ API Connection Test (model={model_to_use}):")
        print(response.choices[0].message["content"])

    except Exception as e2:
        print(f"✗ Error: {e2}")
        print("Please check your API key and model access.")


✓ API Connection Test (model=gpt-4):
API connection Successful

✓ Setup complete! Ready to build the agent.


In [16]:

# Stage 2: Component 1 - LLM and Instructions
# Building a basic agent that can answer questions


In [8]:
class Agent:
    """ A simple AI agent that can answer questions"""

    def __init__(self, api_key):
        """
            Initialize the agent with OpenAI client
            
            Args:
                api_key (str): Your OpenAI API key
        """
        
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4"
        self.system_message = (
            "You are a helpful assistant that breaks down problems into steps "
            "and provides clear explanations."
        )
        self.messages = [] # store conversation memory

    def chat(self, message):
        
        """
        Process a user message and return a response
        
        Args:
            message (str): The user's input message
            
        Returns:
            response object from OpenAI API
        """
        self.messages.append({"role": "user", "content": message})

        response = self.client.chat.completions.create(
           model=self.model,
           messages=[
               {"role": "system", "content": self.system_message},
               *self.messages
               # {"role": "user", "content": message}
           ],
           max_tokens=512,
           temperature=0.1
       )

        assistant_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_message})
        return assistant_message



# ============================================
# TESTING THE BASIC AGENT
# ============================================


# Test 1: Create an agent instance
print("Creating Agent .................")
agent = Agent(api_key=api_key)
print("✓ Agent created!\n")


# Example test
user_input = "Explain Newton's First law."
reply = agent.chat(user_input)
print("Agent reply:\n", reply)

# Test 1
print("Test 1:")
print("User: I have 4 apples. How many do you have?")
reply = agent.chat("I have 4 apples. How many do you have?")
print("Agent:", reply, "\n")

# Test 2
print("Test 2:")
print("User: I ate 1 apple. How many are left?")
reply = agent.chat("I ate 1 apple. How many are left?")
print("Agent:", reply, "\n")

# Test 3
print("Test 3:")
print("User: If I buy 5 more apples, how many will I have in total?")
reply = agent.chat("If I buy 5 more apples, how many will I have in total?")
print("Agent:", reply, "\n")

# Bonus: Show memory
print("=" * 60)
print("BONUS: Conversation memory content:")
for i, msg in enumerate(agent.messages, 1):
    print(f"{i}. {msg['role'].upper()}: {msg['content'][:100]}...")
print("=" * 60)
print("✓ Memory is working correctly!")


Creating Agent .................
✓ Agent created!

Agent reply:
 Sure, I'd be happy to explain Newton's First Law of Motion.

Newton's First Law, also known as the Law of Inertia, states that an object at rest will stay at rest, and an object in motion will stay in motion, unless acted upon by an external force. This law essentially describes how an object's motion will not change unless a force causes it to change.

Let's break it down into steps:

1. **Objects at Rest**: According to Newton's First Law, an object that is not moving (at rest) will continue to stay at rest. For example, if you place a book on a table, it will stay there indefinitely unless something causes it to move, like a gust of wind or someone picking it up.

2. **Objects in Motion**: Similarly, an object that is moving will continue to move in the same direction and at the same speed (this is called maintaining a constant velocity), unless something causes it to do otherwise. For example, if you slide a puck on a

In [ ]:
from openai import OpenAI
import json
import os


class CalculatorTool:
    """
    A tool that performs mathematical calculations.
    This is like giving the agent a calculator it can use!
    """

    def get_schema(self):
        """
        Returns the tool definition in OpenAI's expected format.
        This tells the AI:
        - What the tool is called
        - What it does
        - What parameters it needs
        """
        return {
            "type": "function",
            "function": {
                "name": "calculator",
                "description": (
                    "Performs mathematical calculations. "
                    "Use this for arithmetic operations like addition, "
                    "subtraction, multiplication, division, etc."
                ),
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": (
                                "The mathematical expression to evaluate "
                                "(e.g., '2+2', '157.09 * 493.89', '(10 + 5) / 3')"
                            ),
                        }
                    },
                    "required": ["expression"],
                },
            },
        }

    def execute(self, expression):
        """
        Actually performs the calculation.
        Args:
            expression (str): Math expression like "2+2" or "157.09 * 493.89"
        Returns:
            dict: Result or error message
        """
        try:
            result = eval(expression)
            return {"result": result}
        except Exception as e:
            return {"error": f"Invalid expression: {str(e)}"}


# ============================================
# AGENT CLASS
# ============================================
class Agent:
    """An AI agent that can use tools (like a calculator) to answer questions"""

    def __init__(self, api_key, tools=None):
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o-mini"
        self.system_message = (
            "You are a helpful assistant that breaks down problems into steps "
            "and provides clear explanations."
        )
        self.messages = []
        self.tools = tools if tools else []
        self.tool_map = {tool.get_schema()["function"]["name"]: tool for tool in self.tools}

    def _get_tool_schemas(self):
        """Return all tool schemas in OpenAI format."""
        return [tool.get_schema() for tool in self.tools]

    def chat(self, message,max_iterations=10):
        """Process a user message and return a response (with tool awareness)."""
        self.messages.append({"role": "user", "content": message})
        iteration = 0


        while iteration < max_iterations:
            iteration += 1
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "system", "content": self.system_message}, *self.messages],
                tools=self._get_tool_schemas() if self.tools else None,
                max_tokens=1024,
                temperature=0.1
            )

            assistant_message = response.choices[0].message

            if assistant_message.tool_calls:
                # Store assistant tool request
                self.messages.append({
                    "role": "assistant",
                    "content": assistant_message.content,
                    "tool_calls": assistant_message.tool_calls
                })

                # Execute tools
                for tool_call in assistant_message.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)
                    tool = self.tool_map[tool_name]
                    tool_result = tool.execute(**tool_args)
                    self.messages.append({
                        "role": "tool",
                        "content": json.dumps(tool_result),
                        "tool_call_id": tool_call.id
                    })
            else:
                # Final answer
                self.messages.append({"role": "assistant", "content": assistant_message.content})
                return assistant_message.content

        return "⚠️ Reached maximum iterations."

    


# ============================================
  # HELPER FUNCTION TO RUN AGENT
# ============================================

def run_agent(user_input):
    calculator = CalculatorTool()
    agent = Agent(api_key=api_key, tools=[calculator])
    print(f"\n User Question: {user_input}")
    final_answer = agent.chat(user_input)
    print("\n Final Answer:", final_answer)
    return final_answer


# ============================================
# TESTS
# ============================================

print("\n COMPLETE AGENT WITH TOOL USE AND MEMORY ")

# Simple calculation
run_agent("What is 120.09 * 623.09?")

# Multi-step calculation
run_agent(
    "If my brother is 32 years younger than my mother, and my mother is 30 years older than me, and I am 20, how old is my brother?"
)

# No tool needed
run_agent("What is the capital of France?")



🎉 COMPLETE AGENT WITH TOOL USE AND MEMORY 🎉

📝 User Question: What is 120.09 * 623.09?

🎯 Final Answer: The result of \( 120.09 \times 623.09 \) is \( 74826.8781 \).

📝 User Question: If my brother is 32 years younger than my mother, and my mother is 30 years older than me, and I am 20, how old is my brother?

🎯 Final Answer: Let's break down the problem step by step:

1. **Your age**: You are 20 years old.
2. **Your mother's age**: Since your mother is 30 years older than you, we calculate her age as follows:
   \[
   \text{Mother's age} = \text{Your age} + 30 = 20 + 30 = 50 \text{ years old}
   \]
3. **Your brother's age**: Your brother is 32 years younger than your mother, so we calculate his age as follows:
   \[
   \text{Brother's age} = \text{Mother's age} - 32 = 50 - 32 = 18 \text{ years old}
   \]

Therefore, your brother is 18 years old.

📝 User Question: What is the capital of France?

🎯 Final Answer: The capital of France is Paris.


'The capital of France is Paris.'